### Практическая часть
1. Нидлман Вунш (3 балла)  
Реализуйте алгоритм Нидлмана Вунша для выравнивания последовательностеей. 
На вход принимается две строки, матрица замен и стоимость гэпа. 
В результате верните оптимальное выравнивание и его вес. При проверке 
помните, что оптимальных выравниваний может быть несколько, 
но вес у них должен совпадать.

In [1]:
from Bio.Align import substitution_matrices

In [9]:
def global_alignment(str1, str2, substitution_mtrx, gap_penalty):
    dp = [[0 for _ in range(len(str2) + 1)] for _ in range(len(str1) + 1)]

    for i in range(len(str1) + 1):
        dp[i][0] = i * gap_penalty

    for j in range(len(str2) + 1):
        dp[0][j] = j * gap_penalty

    for i in range(1, len(str1) + 1):
        for j in range(1, len(str2) + 1):
            match = dp[i - 1][j - 1] + substitution_mtrx[str1[i - 1]][str2[j - 1]]
            delete = dp[i - 1][j] + gap_penalty
            insert = dp[i][j - 1] + gap_penalty

            dp[i][j] = max(match, delete, insert)

    final_weight = int(dp[len(str1)][len(str2)])

    aligned_str1, aligned_str2 = "", ""

    i, j = len(str1), len(str2)

    while i > 0 or j > 0:
        if i > 0 and j > 0 and dp[i][j] == dp[i - 1][j - 1] + substitution_mtrx[str1[i - 1]][str2[j - 1]]:
            aligned_str1 = str1[i - 1] + aligned_str1
            aligned_str2 = str2[j - 1] + aligned_str2
            i -= 1
            j -= 1

        elif i > 0 and dp[i][j] == dp[i - 1][j] + gap_penalty:
            aligned_str1 = str1[i - 1] + aligned_str1
            aligned_str2 = "-" + aligned_str2
            i -= 1

        else:
            aligned_str1 = "-" + aligned_str1
            aligned_str2 = str2[j - 1] + aligned_str2
            j -= 1

    return aligned_str1, aligned_str2, final_weight


In [10]:
str1 = "GATT"
str2 = "GCATG"
nuc_matrix = substitution_matrices.load("NUC.4.4")
gap_penalty = -2

aligned_str1, aligned_str2, final_weight = global_alignment(str1, str2, nuc_matrix, gap_penalty)

print(aligned_str1)
print(aligned_str2)
print(final_weight)


G-ATT
GCATG
9


In [ ]:
str1 = "AGTACGCA"
str2 = "TATGC"
gap_penalty = -2

aligned_str1, aligned_str2, final_weight = global_alignment(str1, str2, nuc_matrix, gap_penalty)

print(aligned_str1)
print(aligned_str2)
print(final_weight)


AGTACGCA
--TATGC-
10


In [15]:
str1 = "ACGTACGT"
str2 = "ACGTGCGT"
gap_penalty = -2

aligned_str1, aligned_str2, final_weight = global_alignment(str1, str2, nuc_matrix, gap_penalty)
print(aligned_str1)
print(aligned_str2)
print(final_weight)

ACGTACGT
ACGTGCGT
31


In [16]:
str1 = "ACGT"
str2 = "ACT"
gap_penalty = -2

aligned_str1, aligned_str2, final_weight = global_alignment(str1, str2, nuc_matrix, gap_penalty)
print(aligned_str1)
print(aligned_str2)
print(final_weight)

ACGT
AC-T
13


In [17]:
str1 = "ACGTACGTACGT"
str2 = "ACGACG"
gap_penalty = -2

aligned_str1, aligned_str2, final_weight = global_alignment(str1, str2, nuc_matrix, gap_penalty)
print(aligned_str1)
print(aligned_str2)
print(final_weight)

ACGTACGTACGT
----ACG-ACG-
18


2. Афинные гэпы (4 балла)  
Реализуйте выравнивание с афинными гэпами, алгоритм на 
вход принимает две строки, матрицу замен, штраф за начало 
гэпа α, и за его продолжение β. В результате возвращает 
выравнивание и его вес. Сложность алгоритма квадратичная 
по памяти и по времени.

In [21]:
def global_affine_alignment(str1, str2, substitution_mtrx, alpha, beta):
    rows = len(str1) + 1
    cols = len(str2) + 1

    dp = [[0 for _ in range(cols)] for _ in range(rows)]
    upper = [[float('-inf') for _ in range(cols)] for _ in range(rows)]
    lower = [[float('-inf') for _ in range(cols)] for _ in range(rows)]

    for i in range(1, rows):
        dp[i][0] = alpha + i * beta
        upper[i][0] = alpha + i * beta

    for j in range(1, cols):
        dp[0][j] = alpha + j * beta
        lower[0][j] = alpha + j * beta

    for i in range(1, rows):
        for j in range(1, cols):
            match_score = substitution_mtrx[str1[i - 1]][str2[j - 1]]
            match = dp[i - 1][j - 1] + match_score
            upper[i][j] = max(upper[i - 1][j] + beta, dp[i - 1][j] + alpha + beta)
            lower[i][j] = max(lower[i][j - 1] + beta, dp[i][j - 1] + alpha + beta)
            dp[i][j] = max(match, upper[i][j], lower[i][j])

    aligned_str1 = ''
    aligned_str2 = ''
    i = len(str1)
    j = len(str2)
    x = 'm'

    while i > 0 or j > 0:
        if x == 'm':
            current_match_score = substitution_mtrx[str1[i - 1]][str2[j - 1]]

            if i > 0 and j > 0 and dp[i][j] == dp[i - 1][j - 1] + current_match_score:
                aligned_str1 = str1[i - 1] + aligned_str1
                aligned_str2 = str2[j - 1] + aligned_str2
                i -= 1
                j -= 1

            elif i > 0 and dp[i][j] == upper[i][j]:
                x = 'u'
            elif j > 0 and dp[i][j] == lower[i][j]:
                x = 'l'

        if x == 'l':
            if lower[i][j] == lower[i][j - 1] + beta:
                aligned_str1 = "-" + aligned_str1
                aligned_str2 = str2[j - 1] + aligned_str2
                j -= 1
                x = 'l'
            if lower[i][j] == dp[i][j - 1] + alpha + beta:
                aligned_str1 = "-" + aligned_str1
                aligned_str2 = str2[j - 1] + aligned_str2
                j -= 1
                x = 'm'

        if x == 'u':
            if upper[i][j] == upper[i - 1][j] + beta:
                aligned_str2 = "-" + aligned_str2
                aligned_str1 = str1[i - 1] + aligned_str1
                i -= 1
                x = 'u'

            if upper[i][j] == dp[i - 1][j] + alpha + beta:
                aligned_str2 = "-" + aligned_str2
                aligned_str1 = str1[i - 1] + aligned_str1
                i -= 1
                x = 'm'

    final_weight = dp[len(str1)][len(str2)]

    return aligned_str1, aligned_str2, final_weight

In [25]:
str1 = "PTIMWAYSKPVSNGIMFLILLTWCRCYIRPSNFHMRTQNWTFKVNQLLPLDKYSCMWGIQGMYASTWLSHGCVRVWQYPVM"
str2 = "PFGKWTHLIVSNGIYNSQWTTHHNFQESLELLTWCRCYIRPSMRTQNWTFKVNQLLPLDKYSCMWKIQGMYASTLSHGCVRVWQYPVM"
ac_mtrx = substitution_matrices.load('BLOSUM62')
alpha = -10
beta = -1

aligned_str1, aligned_str2, final_weight = global_affine_alignment(str1, str2, ac_mtrx, alpha, beta)
print(aligned_str1)
print(aligned_str2)
print(final_weight)

PTIMWAYSKPVSNGIM------------FLILLTWCRCYIRPSNFHMRTQNWTFKVNQLLPLDKYSCMWGIQGMYASTWLSHGCVRVWQYPVM
PFGKWTH-LIVSNGIYNSQWTTHHNFQESLELLTWCRCYIRPS---MRTQNWTFKVNQLLPLDKYSCMWKIQGMYAST-LSHGCVRVWQYPVM
302.0


In [26]:
str1 = "HEAGAWGHEE"
str2 = "PAWHEAE"
ac_mtrx = substitution_matrices.load('BLOSUM62')
alpha = -11
beta = -1

aligned_str1, aligned_str2, final_weight = global_affine_alignment(str1, str2, ac_mtrx, alpha, beta)
print(aligned_str1)
print(aligned_str2)
print(final_weight)

HEAGAWGHEE
---PAWHEAE
1.0


In [27]:
str1 = "KGDPKK"
str2 = "RGDPRK"
ac_mtrx = substitution_matrices.load('BLOSUM62')
alpha = -12
beta = -2

aligned_str1, aligned_str2, final_weight = global_affine_alignment(str1, str2, ac_mtrx, alpha, beta)
print(aligned_str1)
print(aligned_str2)
print(final_weight)

KGDPKK
RGDPRK
28.0


In [28]:
str1 = "MKLFA"
str2 = "MKLA"
ac_mtrx = substitution_matrices.load('BLOSUM62')
alpha = -12
beta = -2

aligned_str1, aligned_str2, final_weight = global_affine_alignment(str1, str2, ac_mtrx, alpha, beta)
print(aligned_str1)
print(aligned_str2)
print(final_weight)

MKLFA
MKL-A
4.0


### Теоретическая часть
1. Количество выравниваний (4 балла)  
Выведите рекуррентную формулу количества всех возможных 
выравниваний последовательностей длины *n* и *m* пользуясь 
разбиением всех выравниваний на непересекающиеся 
блоки. (1.5 балл)  
Получите точную формулу, основываясь на начальные условия и 
рекуррентную формулу. (1.5 балл)  
Воспользуйтесь приближением Стирлинга чтобы получить 
приближенную формулу количества выравниваний. (1)

**Решение**

**1. Рекуррентная формула количества выравниваний**

Пусть A(n,m) — количество всех возможных выравниваний последовательностей длины n и m. 

В каждую клетку в таблице выравнивания мы могли прийти тремя различными способами:
- символ из первой последовательности выравнять с символом из второй  - match/mismatch
- символ из первой последовательности выравнять с пропуском во второй - insertion
- пропуск в первой последовательности выравнять с символом из второй - gap

Тогда можно записать рекуррентное соотношение:
A(n, m) = A(n-1, m-1) + A(n-1, m) + A(n, m-1), где

- A(n-1, m-1) - число выравниваний, где перед текущей позицией находится match/mismatch
- A(n-1,m) — количество выравниваний, где перед текущей позицией находится insertion
- A(n,m-1) — количество выравниваний, где перед текущей позицией находится gap

**2. Точная формула**

A(n,m) — количество всех возможных выравниваний можно понимать как количество способов дойти из точки (0,0) в точку (n,m) на решетке, используя три типа шагов: горизонтальный (1,0) gap, вертикальный шаг (0,1) inserion, диагональный шаг (1,1)

Любой такой путь состоит из n «горизонтальных приращений» и m «вертикальных приращений» (так как в итоге мы должны сдвинуться на n вправо и на m вверх). 

Некоторые приращения «горизонталь + вертикаль» мы можем совместить в один диагональный шаг (1,1). 

Пусть k — число диагональных шагов, которые мы используем в пути. Тогда: каждый диагональный шаг «съедает» сразу и одно горизонтальное, и одно вертикальное приращение. Если было израсходовано k диагональных шагов, то остается:
- (n - k) горизонтальных шагов (1,0)
- (m - k) вертикальных» шагов (0,1)

Суммарное число шагов в таком пути: $(n - k) \;+\;(m - k) \;+\;k \;=\; n + m - k$
Из них $(n - k)$ будут горизонтальными $(m - k)$ — вертикальными, а $k$ — диагональными.

Так как $k$ может быть от 0 до $\min(n,m)$, суммируем по всем $k$:

$$A(n,m)
\;=\;
\sum_{k=0}^{\min(n,m)}
\frac{(n + m - k)!}{(n-k)!\,\,(m-k)!\,\,k!}. = \sum_{k=0}^{\min(n,m)}
\binom{n}{k}\,\binom{m}{k}\,2^k$$



**3. Приближённая формула**

$$n!\;\sim\;\sqrt{2\pi n}\,\left(\frac{n}{e}\right)^{n}$$


Каждое слагаемое можно аппроксимировать как:
$\binom{n}{k}
\;\approx\;
\frac{\sqrt{2\pi\,n}\,\bigl(\frac{n}{e}\bigr)^n}
{\sqrt{2\pi\,k}\,\bigl(\frac{k}{e}\bigr)^k \;\sqrt{2\pi\,(n-k)}\,\bigl(\frac{n-k}{e}\bigr)^{\,n-k}}
$

Основная экспоненциальная составляющая асимптотического роста числа выравниваний определяется функцией $3^n$